# This is a full pipeline with keypoints prediction using VRNN in reconstruction mode for VoxCeleb dataset

# Import functions

In [1]:
import os   
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from Training_Prediction.FOMM.Source_Model.logger import Logger, Visualizer
import numpy as np
import imageio
from Training_Prediction.FOMM.Source_Model.sync_batchnorm import DataParallelWithCallback
from Training_Prediction.FOMM.Source_Model.modules.RNN_prediction_module import PredictionModule
from Training_Prediction.FOMM.Source_Model.augmentation import SelectRandomFrames, SelectFirstFrames_two, VideoToTensor
from tqdm import trange
from torch.utils.data import DataLoader, Dataset
from Training_Prediction.FOMM.Source_Model.frames_dataset import FramesDataset
import tensorflow.compat.v1 as tf
from Training_Prediction.PREDICTOR.Source_Model.VRNN import build_vrnn, get_config
import pickle
from Training_Prediction.PREDICTOR.Source_Model.VRNN_prediction import VRNN_predict
from Training_Prediction.PREDICTOR.Source_Model.prediction_toplevel import KPDataset,get_data_from_dataloader_60

import gc
import pickle
import yaml
from Training_Prediction.FOMM.Source_Model.modules.generator import OcclusionAwareGenerator,calculate_frechet_distance,compute_fvd
from Training_Prediction.FOMM.Source_Model.modules.keypoint_detector import KPDetector
from Training_Prediction.FOMM.Source_Model.logger import Logger, Visualizer, Visualizer_slow
from torch import nn

import tensorflow.compat.v1 as tf

import os, sys
os.environ["CUDA_VISIBLE_DEVICES"]='0'

2024-07-12 02:01:13.648356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-12 02:01:14.218799: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Import keypoints of 3883 training videos

In [2]:
with open("kp_train_3883_vox.pkl", "rb") as f:
    kp_time_series = pickle.load(f)
len(kp_time_series)

UnpicklingError: pickle data was truncated

# Convert list of keypoints to dictionary

In [2]:
# convert list of keypoints to dictionary: 
for video_idx in range(len(kp_time_series)):
    kp_time_series[video_idx] = kp_time_series[video_idx]['kp']

kp_dict_init = []
for video_idx in range(len(kp_time_series)): # 
    init_mean = []
    init_jacobian = []
    for frame_idx in range(len(kp_time_series[video_idx])):
        kp_mean = kp_time_series[video_idx][frame_idx]['value'].reshape(1,10,2)
        kp_mean = torch.tensor(kp_mean)
        kp_jacobian = kp_time_series[video_idx][frame_idx]['jacobian'].reshape(1,10,2,2)
        kp_jacobian = torch.tensor(kp_jacobian)

        init_mean.append(kp_mean)
        init_jacobian.append(kp_jacobian)

    init_mean = torch.cat(init_mean)
    init_jacobian = torch.cat(init_jacobian)

    init_mean = torch.reshape(init_mean,(1,init_mean.shape[0],init_mean.shape[1],init_mean.shape[2]))
    init_jacobian = torch.reshape(init_jacobian,(1,init_jacobian.shape[0],10,2,2))

    if torch.cuda.is_available():
        # add tensor to cuda
        init_mean = init_mean.to('cuda:0')
        init_jacobian = init_jacobian.to('cuda:0')

    kp_dict_both = {"value":init_mean,"jacobian":init_jacobian}
    kp_dict_init.append(kp_dict_both)

NameError: name 'kp_time_series' is not defined

# Apply min-max standardization to keypoints 

In [ ]:
kp_list_train = []
for video_idx in range(len(kp_dict_init)):
    kp_one_video = torch.cat((kp_dict_init[video_idx]['value'], kp_dict_init[video_idx]['jacobian'].reshape(1,-1,10,4)),dim=-1).reshape(-1,60)
    kp_one_video_array = np.array(kp_one_video.cpu())
    kp_list_train.append(kp_one_video_array)
    
#####  min-max std to 60 dimensions of selected one video ######
kp_list_train_std = []
min_list = []
range_list = []
for video_idx in range(len(kp_list_train)):
    min_values = np.min(kp_list_train[video_idx],axis=0) # 60 mins of one selected video in the loop
    max_values = np.max(kp_list_train[video_idx],axis=0) # 60 maxs of one selected video in the loop
    range_values = max_values - min_values 
    kp_one_video_std = (kp_list_train[video_idx] - min_values) / range_values
    kp_list_train_std.append(kp_one_video_std)
    min_list.append(min_values)
    range_list.append(range_values)

trajs = kp_list_train_std
print(len(trajs))
print(trajs[0].shape)

# Convert standardized keypoints to mini-batches: 12 or 24 frames a batch

In [ ]:
######### convert data into batches #########
data_batch_train = []

frames = 24 # 24 as one batch, use 12 ground truth frames as input to predict next 12 frames as output
input_frames = int(frames / 2)
input_dim = 60
for t,x in enumerate(kp_list_train_std):
    if x.shape[0] >= frames:
        num_full_batches = x.shape[0] // frames
        for arr in np.array_split(x[:num_full_batches * frames], num_full_batches):
            data_batch_train.append(arr)
print(f'train dataset batches:', len(data_batch_train))
print(data_batch_train[0].shape)

In [ ]:
##### train dataset:

train_data_reshape = np.array(data_batch_train).reshape(-1,frames,60)
train_data_reshape.shape

#  Train VRNN with build_vrnn_new/build_vrnn

In [ ]:
model = build_vrnn

print("using VRNN")
train_data = train_data_reshape

frames = 24
data_all = {}
data_all['value'] = []
data_all['jacobian'] = []
data_lee = []

#data_lee = get_data_50(test_dataset)
data_lee = train_data
print(data_lee.shape)
data_lee = np.reshape(data_lee,(-1,frames,10,6))
print(data_lee.shape)
data_lee = tf.convert_to_tensor(data_lee)
print(data_lee.shape)
cfg = get_config()
input_keypoint = tf.keras.Input(shape=[frames,10,6],name='keypoint')
observed_keypoints_stop = tf.keras.layers.Lambda(tf.stop_gradient)(
input_keypoint)
vrnn_model = model(cfg)
predicted_keypoints, kl_divergence = vrnn_model(observed_keypoints_stop)
train_model = tf.keras.Model(inputs=[input_keypoint],outputs=[predicted_keypoints])
vrnn_coord_pred_loss = tf.nn.l2_loss(
observed_keypoints_stop - predicted_keypoints)
# Normalize by batch size and sequence length:
vrnn_coord_pred_loss /= tf.to_float(
  tf.shape(input_keypoint)[0] * tf.shape(input_keypoint)[1])
train_model.add_loss(vrnn_coord_pred_loss)
kl_loss = tf.reduce_mean(kl_divergence)  # Mean over batch and timesteps.
train_model.add_loss(cfg.kl_loss_scale * kl_loss)
vrnn_optimizer = tf.keras.optimizers.Adam(lr=cfg.learning_rate, clipnorm = cfg.clipnorm)
train_model.compile(vrnn_optimizer)
# Specify the checkpoint path
checkpoint_path = "Checkpoints/VRNN_3883videos_vox_12-12.ckpt" # 15-15 frames

checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback to save the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                             save_weights_only=True,
                                             verbose=1)

print("Training VRNN_LEE prediction...")
train_model.fit(x=data_lee, steps_per_epoch = train_data.shape[0], epochs=10, callbacks=[cp_callback]) 

# After VRNN is trained, load the saved model

In [ ]:
model = build_vrnn
frames = 24
cfg = get_config()
input_keypoint = tf.keras.Input(shape=[frames,10,6],name='keypoint')
observed_keypoints_stop = tf.keras.layers.Lambda(tf.stop_gradient)(
input_keypoint)
vrnn_model = model(cfg)
predicted_keypoints, kl_divergence = vrnn_model(observed_keypoints_stop)
train_model = tf.keras.Model(inputs=[input_keypoint],outputs=[predicted_keypoints])
vrnn_coord_pred_loss = tf.nn.l2_loss(
observed_keypoints_stop - predicted_keypoints)
# Normalize by batch size and sequence length:
vrnn_coord_pred_loss /= tf.to_float(
  tf.shape(input_keypoint)[0] * tf.shape(input_keypoint)[1])
train_model.add_loss(vrnn_coord_pred_loss)
kl_loss = tf.reduce_mean(kl_divergence)  # Mean over batch and timesteps.
train_model.add_loss(cfg.kl_loss_scale * kl_loss)

# Load saved model:
cfg = get_config()
checkpoint_path = "Checkpoints/VRNN_3883videos_vox_12-12.ckpt" 
# checkpoint_path = "Checkpoints/VRNN_3883videos_vox_6-6.ckpt" 

# Loads the weights
train_model.load_weights(checkpoint_path)
train_model.reset_states()

# Import keypoints of 44 VoxCeleb test videos

In [ ]:
with open("kp_test_44_vox.pkl", "rb") as f:
    kp_time_series = pickle.load(f)
len(kp_time_series)

# Convert list of keypoints to dictionary

In [4]:
for video_idx in range(len(kp_time_series)):
    kp_time_series[video_idx] = kp_time_series[video_idx]['kp']

kp_dict_init = []
for video_idx in range(len(kp_time_series)): # 
    init_mean = []
    init_jacobian = []
    for frame_idx in range(len(kp_time_series[video_idx])):
        kp_mean = kp_time_series[video_idx][frame_idx]['value'].reshape(1,10,2)
        kp_mean = torch.tensor(kp_mean)
        kp_jacobian = kp_time_series[video_idx][frame_idx]['jacobian'].reshape(1,10,2,2)
        kp_jacobian = torch.tensor(kp_jacobian)

        init_mean.append(kp_mean)
        init_jacobian.append(kp_jacobian)

    init_mean = torch.cat(init_mean)
    init_jacobian = torch.cat(init_jacobian)

    init_mean = torch.reshape(init_mean,(1,init_mean.shape[0],init_mean.shape[1],init_mean.shape[2]))
    init_jacobian = torch.reshape(init_jacobian,(1,init_jacobian.shape[0],10,2,2))

    if torch.cuda.is_available():
        # add tensor to cuda
        init_mean = init_mean.to('cuda:0')
        init_jacobian = init_jacobian.to('cuda:0')

    kp_dict_both = {"value":init_mean,"jacobian":init_jacobian}
    kp_dict_init.append(kp_dict_both)

# Apply min-max std to keypoints and convert to batches

In [5]:
kp_list_test = []
for video_idx in range(len(kp_dict_init)):
    kp_one_video = torch.cat((kp_dict_init[video_idx]['value'], kp_dict_init[video_idx]['jacobian'].reshape(1,-1,10,4)),dim=-1).reshape(-1,60)
    kp_one_video_array = np.array(kp_one_video.cpu())
    kp_list_test.append(kp_one_video_array)
    
#####  min-max std to 60 dimensions of selected one video ######
kp_list_test_std = []
min_list = []
range_list = []
for video_idx in range(len(kp_list_test)):
    data = kp_list_test[video_idx]
    data_length = len(kp_list_test[video_idx])
    step_interval = 12 # choose between 12 frames or 24 frames 
    min_required_steps = 2*step_interval
    selected_data = []
    for i in range(0, data_length - min_required_steps+1, 2 * step_interval):
        selected_data.extend(data[i:i + step_interval])
    min_values = np.min(selected_data,axis=0) # 60 mins of one selected video in the loop
    max_values = np.max(selected_data,axis=0) # 60 maxs of one selected video in the loop 
    range_values = max_values - min_values 
    kp_one_video_std = (kp_list_test[video_idx] - min_values) / range_values
    kp_list_test_std.append(kp_one_video_std)
    min_list.append(min_values)
    range_list.append(range_values)

trajs = kp_list_test_std
print(len(trajs))
print(trajs[0].shape)

44
(118, 60)


In [6]:
######### convert into batches:
frames = min_required_steps
input_frames = int(frames / 2)
data_batch_test = []
for t,x in enumerate(kp_list_test_std):
    if x.shape[0] >= frames:
        num_full_batches = x.shape[0] // frames
        for arr in np.array_split(x[:num_full_batches * frames], num_full_batches):
            data_batch_test.append(arr)
print(f'test dataset batches:', len(data_batch_test))
print(data_batch_test[0].shape)

test dataset batches: 529
(24, 60)


In [7]:
###### test dataset:

test_data_reshape = np.array(data_batch_test).reshape(-1,frames,60)
test_data_reshape.shape

(529, 24, 60)

# Predict keypoints using trained model:

In [8]:
# Load test dataset and process model.predict():

validation_data = test_data_reshape

validation_data_tensor = tf.convert_to_tensor(validation_data.reshape(-1,frames,10,6))
pred = train_model.predict(validation_data_tensor)
print(pred.shape)

 9/17 [==============>...............] - ETA: 0s

2024-06-08 01:15:25.323035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


17/17 [==============================] - 5s 15ms/step
(529, 24, 10, 6)


# Generate unstd keypoints:

In [9]:
# save num_batches for each video:
num_batch_video = []
num_full_batches_all = 0
for t,x in enumerate(kp_list_test_std):
    if x.shape[0] >= frames:
        num_full_batches = x.shape[0] // frames
        num_full_batches_all += num_full_batches
        num_batch_video.append(num_full_batches)
print(f'number of batches of each video:', len(num_batch_video))

number of batches of each video: 44


In [10]:
# first half of frames: groundtruth; last half of frames: predicted
test_gt_pred = np.concatenate((test_data_reshape[:,:input_frames], pred.reshape(-1,frames,60)[:,input_frames:]), axis = 1)
test_gt_pred.shape

(529, 24, 60)

In [11]:
# unstd for each video:
test_video_unstd_list = []
for video_idx in range(len(num_batch_video)):
    test_video = test_gt_pred[sum(num_batch_video[:video_idx]):sum(num_batch_video[:video_idx+1])]
    test_video_unstd = test_video * range_list[video_idx] + min_list[video_idx]
    test_video_unstd_list.append(test_video_unstd) # unstd video keypoints

# Optical flow and generator

In [23]:
####### call the config functions and inference dataloader #########
config="config/abs-vox.yml"

# Test dataset
with open(config) as f:
    config = yaml.safe_load(f)
dataset = FramesDataset(is_train=(False), **config['dataset_params'],mode="RNN") # test

print(len(dataset))
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=1)

### call the functions        
generator = OcclusionAwareGenerator(**config['model_params']['generator_params'],
                                        **config['model_params']['common_params'])
kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
                            **config['model_params']['common_params'])

log_dir="./log/test-reconstruction-vox"
checkpoint="./Training_Prediction/FOMM/Trained_Models/vox-cpk.pth.tar"

if checkpoint is not None:
    Logger.load_cpk(checkpoint, generator=generator, kp_detector=kp_detector)
else:
    raise AttributeError("Checkpoint should be specified for mode='reconstruction'.")
    
def save_obj(obj, name ):
    with open('./'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('./' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

png_dir = os.path.join(log_dir, 'prediction/png')
log_dir = os.path.join(log_dir, 'prediction')

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

if not os.path.exists(png_dir):
    os.makedirs(png_dir)

if torch.cuda.is_available():
    generator = DataParallelWithCallback(generator)
    kp_detector = DataParallelWithCallback(kp_detector)

generator.eval()
kp_detector.eval()

prediction_params = config['prediction_params']

num_epochs = prediction_params['num_epochs']
lr = prediction_params['lr']
bs = prediction_params['batch_size']
num_frames = prediction_params['num_frames']
loss_list_total = []
fvd_list_total = []

Use predefined train-test split.
using videos from test directory
['id10280#NXjT3732Ekg#001093#001192.mp4', 'id10281#NHARUN9OhSo#000605#000886.mp4', 'id10281#NHARUN9OhSo#001059#001210.mp4', 'id10281#NHARUN9OhSo#002098#002175.mp4', 'id10281#NHARUN9OhSo#002209#002570.mp4', 'id10281#NHARUN9OhSo#006609#006906.mp4', 'id10281#NHARUN9OhSo#006912#007284.mp4', 'id10281#NHARUN9OhSo#007425#007663.mp4', 'id10282#IDA_ElNHLn4#000674#000852.mp4', 'id10282#IDA_ElNHLn4#001226#001390.mp4', 'id10283#N69Hp2DGMLk#000519#000619.mp4', 'id10283#N69Hp2DGMLk#000721#000842.mp4', 'id10283#N69Hp2DGMLk#000893#001589.mp4', 'id10283#N69Hp2DGMLk#004133#005157.mp4', 'id10283#N69Hp2DGMLk#005157#005316.mp4', 'id10283#N69Hp2DGMLk#005931#006184.mp4', 'id10283#N69Hp2DGMLk#006184#006353.mp4', 'id10283#N69Hp2DGMLk#006405#006583.mp4', 'id10283#N69Hp2DGMLk#006600#007118.mp4', 'id10283#N69Hp2DGMLk#007129#007281.mp4', 'id10283#r9-0pljhZqs#002414#002769.mp4', 'id10283#r9-0pljhZqs#003725#003847.mp4', 'id10283#r9-0pljhZqs#004062#004

OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB (GPU 0; 23.68 GiB total capacity; 992.28 MiB already allocated; 26.69 MiB free; 1.11 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [19]:
i = 0
for it, x in tqdm(enumerate(dataloader)):
    if config['reconstruction_params']['num_videos'] is not None:
        if it > config['reconstruction_params']['num_videos']:
            break
    if i >= 1:
        break
    # Clear the PNG directory for each iteration of the outer loop
    shutil.rmtree(png_dir, ignore_errors=True)
    os.makedirs(png_dir)
    
    with torch.no_grad():
        predictions = []
        visualizations = []

        ######## keypoints ########
        kp_driving_video = test_video_unstd_list[it].reshape(-1, 10, 6)
        kp_driving_video = torch.tensor(kp_driving_video)
        kp_source = {"value":kp_driving_video[0,:,:2].reshape(1,10,2),"jacobian":kp_driving_video[0,:,2:].reshape(1,10,2,2)} # kp of the ith frame

        # Start generator
        for i in range(((x['video'].shape[2]) // frames) * frames):  # cut the last <24 frames
            source = x['video'][:, :, 0]
            driving = x['video'][:, :, i]
            kp_driving = {"value": kp_driving_video[i, :, :2], "jacobian": kp_driving_video[i, :, 2:]}  # kp of the ith frame
            kp_driving['value'] = kp_driving['value'].reshape(1, 10, 2)
            kp_driving['jacobian'] = kp_driving['jacobian'].reshape(1, 10, 2, 2)
            out = generator(source, kp_source=kp_source, kp_driving=kp_driving)
            out['kp_source'] = kp_source
            out['kp_driving'] = kp_driving
            del out['sparse_deformed']
            predictions.append(np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0])
            
            # Save each frame as PNG to png_dir
            frame_name = f"frame_{i}.png"
            frame_path = os.path.join(png_dir, frame_name)
            imageio.imsave(frame_path, (255 * out['prediction'].detach().cpu().numpy()).astype(np.uint8))
        i = i + 1
            

0it [00:00, ?it/s]


ValueError: Image must be 2D (grayscale, RGB, or RGBA).

In [2]:
#########  FOMM+VRNN ########
i = 0
for it, x in tqdm(enumerate(dataloader)):
        if config['reconstruction_params']['num_videos'] is not None:
            if it > config['reconstruction_params']['num_videos']:
                break
        if i > 0:
            break
        with torch.no_grad():
            predictions = []
            visualizations = []

            ######## keypoints ########
            kp_driving_video = test_video_unstd_list[it].reshape(-1,10,6)
            kp_driving_video = torch.tensor(kp_driving_video)
            kp_source = {"value":kp_driving_video[0,:,:2].reshape(1,10,2),"jacobian":kp_driving_video[0,:,2:].reshape(1,10,2,2)} # kp of the ith frame      
        
        ##### Start generator
        loss_list = []
        fvd_list = []
        for i in range(((x['video'].shape[2])//frames)*frames): # cut the last <24 frames
            source = x['video'][:, :, 0]
            driving = x['video'][:, :, i]
            kp_driving = {"value":kp_driving_video[i,:,:2],"jacobian":kp_driving_video[i,:,2:]} # kp of the ith frame
            kp_driving['value'] = kp_driving['value'].reshape(1,10,2)
            kp_driving['jacobian'] = kp_driving['jacobian'].reshape(1,10,2,2)
            out = generator(source, kp_source=kp_source, kp_driving=kp_driving)
            out['kp_source'] = kp_source
            out['kp_driving'] = kp_driving
            del out['sparse_deformed']
            predictions.append(np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0])

            visualization = Visualizer(**config['visualizer_params']).visualize(source=source,
                                                                                    driving=driving, out=out)
            visualizations.append(visualization)
            # mse loss
            if np.abs(out['prediction'].detach().cpu().numpy() - driving.cpu().numpy()).mean() != 0:
                loss_list.append(np.abs(out['prediction'].detach().cpu().numpy() - driving.cpu().numpy()).mean())
                # Calculate FVD for each frame using ground truth and predicted videos
                ground_truth_features = driving.detach().cpu().permute(0,2,3,1).reshape(256,256,3)
                predicted_features = out['prediction'].detach().cpu().permute(0,2,3,1).reshape(256,256,3)
                fvd_list.append(compute_fvd(ground_truth_features, predicted_features))

        print("Reconstruction loss: %s" % np.mean(loss_list))
        loss_list_total.append(np.mean(loss_list))

        print("FVD Score: %s" % np.mean(fvd_list))
        fvd_list_total.append(np.mean(fvd_list))

        predictions = np.concatenate(predictions, axis=1)
        imageio.imsave(os.path.join(png_dir, x['name'][0] + '.png'), (255 * predictions).astype(np.uint8))
        image_name = x['name'][0] + config['reconstruction_params']['format']
        imageio.mimsave(os.path.join(log_dir, image_name), visualizations)

print("mean Reconstruction loss: %s" % np.mean(loss_list_total)) 
print("mean FVD score: %s" % np.mean(fvd_list_total)) 

NameError: name 'dataloader' is not defined